In [ ]:
# an attempt on Actor-Critic A2C
# not part of the project as I haven't figure out the correct feedback

# code modified from
# https://github.com/pytorch/examples/blob/master/reinforcement_learning/actor_critic.py

In [ ]:
# TODO
# - implement it
# - test whether sharing first layers help improving performance

In [100]:
%load_ext autoreload
%autoreload 2

import gc
import copy
import random
import numpy as np
import axelrod as axl
from time import time
from pprint import pprint
from itertools import permutations
from collections import namedtuple, deque

np.set_printoptions(precision=3)
pd.options.display.float_format = "{:,.2f}".format

import network
from axl_utils import State, set_match, set_play

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
C = axl.Action.C
D = axl.Action.D

# config game rules
GAME_LEN = 20 + 1
GAME = axl.Game(r=3, s=0, t=5, p=1)
Match = set_match(game=GAME, turns=GAME_LEN)
play = set_play(Match)

game = play(axl.Prober4(), axl.TitForTat())

[(3, 3), (3, 3), (5, 0), (0, 5), (5, 0), (1, 1), (1, 1), (0, 5), (3, 3), (5, 0), (0, 5), (5, 0), (0, 5), (3, 3), (5, 0), (0, 5), (5, 0), (1, 1), (0, 5), (5, 0)]
Player 1 score = 50
Player 2 score = 45


In [3]:
class NNplayer(axl.Player):
    """
    """
    
    name = 'NNplayer'
    classifier = {
        'memory_depth': -1,
        'stochastic': False,
        'inspects_source': False,
        'manipulates_source': False,
        'manipulates_state': False
    }
    
    decision = (axl.Action.C, axl.Action.D)
    
    def __init__(self, network, state, gamma=0.999, mode="dense", N=-1):
        super().__init__()
        
        self.network = network
        self.state   = state
        
        self.gamma   = gamma
        
        self.mode = 1 if mode=="dense" else 0
        self.N = -1
        self.reset()
        
    def reset(self):
        self.state.reset()
        self.reward = 0
        self.network.reset_state()
        
    def strategy(self, opponent):
        """Query the network to make decision"""
        idx = self.network.query(self.state.values())
        return self.decision[idx]
    
    # overwrite update_history to update self state
    # this function is automatically called by axelrod library
    def update_history(self, *args):
        self.history.append(*args)
        self.update_state(*args)
        
    def update_state(self, play, coplay):
        """update current game state & record transition into replay memory"""
        s  = self.state.values()
        s_ = self.state.push(play, coplay)
        
        # reward
        r  = axl.interaction_utils.compute_scores([(play, coplay)])[0][0]
        
        self.network.rewards.append(r)
#         # dense reward
#         if self.mode:
#             r  = r if s[0,0,1]==-1 else np.NaN  # set last turn reward to NaN
#             self.memory.push(s, play, s_, r)
        
#         # sparse reward
#         else:
#             if s[0,0,1]==self.N:
#                 self.memory.push(s, play, s_, 0)
#                 self.reward += r
#             else:
#                 self.memory.push(s, play, s_, r+self.reward)
#                 self.reward = 0
        
    def on_policy_train(self, epoch, param):
        param['t'] = 1
        length = len(self.memory)
        for _ in range(epoch):
            # organize data
            ts = Transition(*zip(*self.memory.sample(length)))
            ss  = np.vstack(ts.state)
            ss_ = np.vstack(ts.next_state)
            ats = np.array([[True, False] if a==axl.Action.C else [False, True] for a in ts.action])
            rs  = np.array(ts.reward, ndmin=2).T
            
            # pass to network
            self.network.learn((ss, ss_, ats, rs), param, self.gamma)
        
        self.network.update_target()
        self.loss = self.network.loss
              
    def off_policy_train(self, param):
        self.network.learn(param)
    
    def plot(self, **kwargs):
        self.network.plot(**kwargs)
        
    # test mode using "with" statement
    def __enter__(self, *args):
        self.network.test_mode(True)
    
    def __exit__(self, *args):
        self.network.test_mode(False)

In [116]:
class A2C():
    
    def __init__(self, actor, critic, gamma=0.9):
        
        self.actor  = actor
        self.critic = critic
        self.gamma = gamma
        self.test = False
        
        self.reset_state()
    
    def reset_state(self):
        self.actions = []  # [(log_prob(chosen action), state_value)]
        self.rewards = []  # [reward from env]
        
    def forward(self, state):
        
        # probability of actions :: 1x[n actions] array
        probs = self.actor(state)[0]
        
        # state value :: 1x1 array
        value = self.critic(state)
        
        return probs, value
    
    def __call__(self, *args):
        return self.forward(*args)
    
    def query(self, state):
        
        probs, value = self.forward(state)
        
        if self.test:
            print(probs)
            return probs.argmax()
        
        # sample action
        cum_probs = np.cumsum(probs)
        action = (cum_probs > np.random.uniform()).argmax()  # Int index of action
        
        # save
        self.actions.append((action, np.log(probs[action]),value))  # (Int, 1x1 array, 1x1 array)
        
        return action
    
    def learn(self, param):
        
        self.actor.set_optimizer(param)
        self.critic.set_optimizer(param)
        self.actor.set_loss_func('mse')
        self.critic.set_loss_func('mse')
        
        # cumulative discounted reward a.k.a "true" value
        returns = []
        cum_r = 0
        for R in self.rewards[::-1]:
            cum_r = R + self.gamma * cum_r
            returns.insert(0, cum_r)
            
        # standardize for better convergence
        returns = (returns - np.mean(returns)) / np.std(returns)
        
        # calculate losses
        policy_losses = []
        value_losses = []
        for (action, log_prob, value), R in zip(self.actions, returns):
            
            advantage = R - value
            policy = np.array([1, 0]) if action==0 else np.array([0,1])  # HARDCODED FOR NOW

            # record losses
            policy_losses.append(log_prob * advantage * policy)  # DOUBLE CHECK THIS !!
            value_losses.append(self.critic.loss_fn(R, value)[0])
        
        # sum all losses then feedback to networks
        policy_loss = np.array(np.sum(policy_losses))
        value_loss  = np.array(np.sum(value_losses))
        print(policy_loss, value_loss)
        self.actor.backprop(policy_loss, param)
        self.critic.backprop(value_loss, param)
        
    def test_mode(self, on):
        if on:
            self.test = True
        else:
            self.test = False
            
    def plot(self):
        pass

In [121]:
layer1 = network.Linear_layer(GAME_LEN*2, 100)

actor = network.NeuralNetwork([
                    network.Flatten_layer(),
                    layer1,
                    network.Activation_layer('ReLU'),
                    network.Linear_layer(100, 40),
                    network.Activation_layer('ReLU'),
                    network.Linear_layer(40, 2),
                    network.Activation_layer('Softmax')
                    ])
critic = network.NeuralNetwork([
                    network.Flatten_layer(),
                    layer1,
                    network.Activation_layer('ReLU'),
                    network.Linear_layer(100, 200),
                    network.Activation_layer('ReLU'),
                    network.Linear_layer(200, 1),
                    ])

p2 = NNplayer(A2C(actor, critic), State(GAME_LEN, C=1, D=-1, N=0.1), gamma=0.9)
param = {"lr": 3e-4, 'batch': 16, "momentum": 0.9, "mode": "train", "eps": 1e-16, "beta":(0.9, 0.999), 
         "epoch": 0, 'optimizer': 'adam', 't': 1, 'clip': 1.0, 'decay': 0.0}

del actor, critic
gc.collect()

0

In [122]:
with p2:
    play(p2, axl.TitForTat())

[0.477 0.523]
[0.526 0.474]
[0.297 0.703]
[0.498 0.502]
[0.356 0.644]
[0.77 0.23]
[0.642 0.358]
[0.51 0.49]
[0.265 0.735]
[0.863 0.137]
[0.537 0.463]
[0.632 0.368]
[0.249 0.751]
[0.178 0.822]
[0.594 0.406]
[0.334 0.666]
[0.375 0.625]
[0.127 0.873]
[0.725 0.275]
[0.368 0.632]
[0.798 0.202]
[(5, 0), (0, 5), (5, 0), (1, 1), (1, 1), (0, 5), (3, 3), (3, 3), (5, 0), (0, 5), (3, 3), (3, 3), (5, 0), (1, 1), (0, 5), (5, 0), (1, 1), (1, 1), (0, 5), (5, 0)]
Player 1 score = 47
Player 2 score = 42


In [ ]:
# run inifinitely many episodes
for i_episode in range(1000):

    play(p2, axl.TitForTat())

    # perform backprop
    p2.off_policy_train(param)

In [126]:
with p2:
    play(p2, axl.TitForTat())

[0.38 0.62]
[0.254 0.746]
[0.32 0.68]
[0.619 0.381]
[0.628 0.372]
[0.327 0.673]
[0.106 0.894]
[0.618 0.382]
[0.753 0.247]
[0.241 0.759]
[0.022 0.978]
[0.336 0.664]
[0.439 0.561]
[0.174 0.826]
[0.841 0.159]
[0.872 0.128]
[0.404 0.596]
[0.037 0.963]
[0.777 0.223]
[0.821 0.179]
[0.03 0.97]
[(5, 0), (1, 1), (1, 1), (0, 5), (3, 3), (5, 0), (1, 1), (0, 5), (3, 3), (5, 0), (1, 1), (1, 1), (1, 1), (1, 1), (0, 5), (3, 3), (5, 0), (1, 1), (0, 5), (3, 3)]
Player 1 score = 40
Player 2 score = 40


In [107]:
p2.network.actor.print_parameters()

--0--
Printing flatten layer:
{'optimizer': <function Layer.set_optimizer.<locals>.optimizer at 0x000001A2FF68F310>,
 'shape': (1, 2, 21),
 'type': 'flatten'}
--1--
Printing linear layer:
{'bias': 0,
 'input': array([[ 0,  1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1,  0,  1,  1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]]),
 'input_nodes': 42,
 'm1': array([[ 0.000e+00,  0.000e+00,  0.000e+00, ...,  0.000e+00,  0.000e+00,
         0.000e+00],
       [ 7.783e-04,  3.708e-70,  5.804e-04, ..., -7.346e-05, -5.291e-04,
        -2.403e-05],
       [-6.379e-03,  1.233e-69, -1.322e-03, ...,  6.213e-05, -1.711e-04,
        -6.586e-04],
       ...,
       [-1.251e-03,  6.910e-70, -1.493e-03, ..., -6.213e-05,  1.711e-04,
        -6.586e-04],
       [-1.251e-03, -1.153e-69, -1.493e-03, ..., -6.213e-05,  1.711e-04,
        -6.586e-04],
       [-1.251e-03, -1.290e-69, -1.493e-03, ..., -6.213e-05,  1.711e-04,
        -6.58